This notebook converts the the logs in [ROS bag format](http://wiki.ros.org/rosbag), that were used to log the events in the application and robot actions, to tables in CSV data format.

Tables are organised as per event type (i.e. per ROS topic) and per participant and exported to CSV files.

In [1]:
import os
import shutil

import pandas as pd
import pathlib as pl
import bagpy

### Define paths.

In [2]:
data_dir = pl.Path('../../data/venue_2')
processed_dir = pl.Path('../processed_data/log_tables')

# Create output directories if they do not exist.
for folder in [processed_dir]:
    if not folder.exists():
        folder.mkdir(parents=True)

### Make a temporary copy of the bag files.
The bagpy module creates tables at the place of the bag file. Therefore, we make a temporary copy of each bag file, then delete them not to modify the original data folder.

In [3]:
# List of all bag files in the data directory.
bag_files = sorted(data_dir.glob('*.bag'))
print('{} bag files found:'.format(len(bag_files)))
for file in sorted(bag_files):
    print(file)

# Make a map for the temporary copies.
temp_map = {}
for f in sorted(bag_files):
    index = f.name.split('_')[0]
    new_name = 'justhink21_log_{:02d}.bag'.format(int(index))
    temp_map[f] = processed_dir.joinpath(new_name)   

# Copy each file to its temporary position.
for original in sorted(temp_map):
    temp = temp_map[original]
    print('Copying {} to {}'.format(original, temp))
    shutil.copy(original, temp)

10 bag files found:
../../data/venue_2/10_2021-06-11_14-31-27.bag
../../data/venue_2/1_2021-06-07_08-50-35.bag
../../data/venue_2/2_2021-06-07_10-23-50.bag
../../data/venue_2/3_2021-06-07_12-42-01.bag
../../data/venue_2/4_2021-06-07_14-27-51.bag
../../data/venue_2/5_2021-06-09_08-41-13.bag
../../data/venue_2/6_2021-06-09_10-10-24.bag
../../data/venue_2/7_2021-06-09_12-43-33.bag
../../data/venue_2/8_2021-06-09_14-30-38.bag
../../data/venue_2/9_2021-06-11_12-41-21.bag
Copying ../../data/venue_2/10_2021-06-11_14-31-27.bag to ../processed_data/log_tables/justhink21_log_10.bag
Copying ../../data/venue_2/1_2021-06-07_08-50-35.bag to ../processed_data/log_tables/justhink21_log_01.bag
Copying ../../data/venue_2/2_2021-06-07_10-23-50.bag to ../processed_data/log_tables/justhink21_log_02.bag
Copying ../../data/venue_2/3_2021-06-07_12-42-01.bag to ../processed_data/log_tables/justhink21_log_03.bag
Copying ../../data/venue_2/4_2021-06-07_14-27-51.bag to ../processed_data/log_tables/justhink21_log_

### Convert bag files to csv files per topic and per participant.

In [4]:
# For each temporary bag file.
for temp_bag_file in sorted(temp_map.values()):
    participant_file = temp_bag_file.with_suffix('.csv')
    print('Reading', str(temp_bag_file))

    # Create a reader for the bag.
    bag = bagpy.bagreader(str(temp_bag_file), verbose=True)

    # Make a csv file for every topic in the bag:
    # bagreader implicitly writes to csv files in a folder.
    csvfiles = []
    for topic in bag.topics:
        file = bag.message_by_topic(topic)
        csvfiles.append(file)

    # Put all the logs of a participant into one table.
    participant_df = pd.DataFrame()
    # For each topic
    for file in csvfiles:
        # Construct a shorter name.
        topic_name = str(file.split('/')[-1].split('.')[0])

        participant_df = participant_df.append(pd.DataFrame(
            [[topic_name], []], columns=["Topic"]))
        df = pd.read_csv(file)

        # Reset time to have duration in the time column.
        df['Time'] -= df.iloc[0]['Time']

        # Drop the header time columns.
        l = ['header.stamp.secs', 'header.stamp.nsecs']
        for col in l:
            if col in df.columns:
                df.drop(col, axis=1, inplace=True)

        # Append the table from each topic into one table.
        participant_df = participant_df.append(df)

    # Make a csv file for the participant (all topics).
    participant_df.to_csv(participant_file, index=False)
    print('Written participant log to {}'.format(participant_file))

Reading ../processed_data/log_tables/justhink21_log_01.bag
[INFO]  Successfully created the data folder ../processed_data/log_tables/justhink21_log_01.
Written participant log to ../processed_data/log_tables/justhink21_log_01.csv
Reading ../processed_data/log_tables/justhink21_log_02.bag
[INFO]  Successfully created the data folder ../processed_data/log_tables/justhink21_log_02.
Written participant log to ../processed_data/log_tables/justhink21_log_02.csv
Reading ../processed_data/log_tables/justhink21_log_03.bag
[INFO]  Successfully created the data folder ../processed_data/log_tables/justhink21_log_03.
Written participant log to ../processed_data/log_tables/justhink21_log_03.csv
Reading ../processed_data/log_tables/justhink21_log_04.bag
[INFO]  Successfully created the data folder ../processed_data/log_tables/justhink21_log_04.
Written participant log to ../processed_data/log_tables/justhink21_log_04.csv
Reading ../processed_data/log_tables/justhink21_log_05.bag
[INFO]  Successfully 

### Delete the temporary bag files.

In [5]:
# remove bag files from log_tables dir
for temp in sorted(temp_map.values()):
    os.remove(str(temp))
    print("Removed temporary bag file {}".format(temp))

Removed temporary bag file ../processed_data/log_tables/justhink21_log_01.bag
Removed temporary bag file ../processed_data/log_tables/justhink21_log_02.bag
Removed temporary bag file ../processed_data/log_tables/justhink21_log_03.bag
Removed temporary bag file ../processed_data/log_tables/justhink21_log_04.bag
Removed temporary bag file ../processed_data/log_tables/justhink21_log_05.bag
Removed temporary bag file ../processed_data/log_tables/justhink21_log_06.bag
Removed temporary bag file ../processed_data/log_tables/justhink21_log_07.bag
Removed temporary bag file ../processed_data/log_tables/justhink21_log_08.bag
Removed temporary bag file ../processed_data/log_tables/justhink21_log_09.bag
Removed temporary bag file ../processed_data/log_tables/justhink21_log_10.bag
